# Gold Prices Time Series Forecasting

### IMPORT LIBRARIES

In [ ]:
import pathlib
import numpy as np
import pandas as pd
from statsmodels.graphics.tsaplots import month_plot
from pandas.plotting import lag_plot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.ar_model import AutoReg
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse 
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_predict
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX 
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import date, datetime
%matplotlib inline

# Ignore harmless warnings 
import warnings 
warnings.filterwarnings("ignore")

### LOAD DATA

In [ ]:
FILE_PATH = pathlib.Path.cwd().joinpath('annual_gold_prices.csv')
df = pd.read_csv(FILE_PATH)

### DESCRIPTIVE OVERVIEW OF THE DATAFRAME

In [ ]:
print("----------------------------------DATAFRAME INFO----------------------------------")
print(df.info())
print("----------------------------------------------------------------------------------")
print("\n" + "\n" + "----------------------------------DATAFRAME HEAD----------------------------------")
print(df.head())
print("----------------------------------------------------------------------------------")
print("\n" + "\n" + "----------------------------------DATAFRAME DESCIPTION----------------------------------")
print(df.describe())
print("----------------------------------------------------------------------------------")
print("\n" + "\n" + "----------------------------------DATAFRAME SHAPE----------------------------------")
print(df.shape)
print("----------------------------------------------------------------------------------")

### PRE-PROCESSING

In [ ]:
df.columns = [col.strip().replace(' ', '_').lower() for col in df.columns]

In [ ]:
df_column_names = list(df.columns.values)
print(df_column_names)

In [ ]:
for col in df.columns:
    print("-------------------------START----------------------------------------")
    #print(col)
    print(df[col].value_counts())
    print("--------------------------END-----------------------------------------" + "\n" + "\n")

### CHECK IF ANY COLUMN HAS MISSING DATA

In [ ]:
#CHECK IF ANY COLUMN HAS MISSING DATA
not_null_df = df.isnull()
print(not_null_df)

df.isnull().sum()

### CHECK AND HANDLE DUPLICATES IF ANY

In [ ]:
#CHECK AND HANDLE DUPLICATES IF ANY
if_duplicate = df[df.duplicated(keep='last')]
if_duplicate.shape

## CLASS DISTRIBUTION WITH AND WITHOUT DUPLICATES
#print("WITH DUPLICATES : ", df.shape)
#df.drop_duplicates(inplace=True)
#print("WITHOUT DUPLICATES : ", df.shape)

### SHOW ALL ROWS

In [ ]:
pd.set_option("display.max_rows",None)

In [ ]:
df

In [ ]:
print(df.date)

In [ ]:
print(df.price)

### RESET TO SHOW DEFAULT NUMBER OF ROWS

In [ ]:
pd.reset_option("display.max_rows")

### EXPLORATORY DATA ANALYSIS

#### UNDERSTANDING THE DATA

In [ ]:
print(f"DATE RANGE OF GOLD PRICES STARTS FROM - {df.loc[:, 'date'][0]} TO {df.loc[:,'date'][len(df)-1]}")

In [ ]:
date = pd.date_range(start='12/1950', end='12/2020', freq='Y')
#date = pd.date_range(start='1/1/1950', end='8/1/2020', freq='M')
date

In [ ]:
len(date)

In [ ]:
df['month'] = date
df.drop('date', axis=1, inplace=True)
df = df.set_index('month')
df.head()

In [ ]:
df.plot(figsize=(15, 8))
plt.title("Annual Gold Prices since 1950")
plt.xlabel("Month")
plt.ylabel("Price")
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
round(df.describe(), 3)

### Inference
#### 1. The Average gold price for the last 60 Years is $412.78

#### 2. Only 25% of the time, the gold is above $458.94

#### 3. Highest Gold price during this time was $1687.34

#### VISUAL ANALYSIS

In [ ]:
_, ax = plt.subplots(figsize=(14, 7))
sns.boxplot(x = df.index.year,y = df.values[:,0], ax=ax)
#sns.boxplot(x=df.values[:,0], y=df.price , ax=ax)
plt.title("Annual Gold Prices since 1950")
plt.xlabel("Year")
plt.ylabel("Price")
plt.xticks(rotation=90)
plt.grid()
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
df

In [ ]:
print(df.price)

In [ ]:
print("---------------------------------YEARS-----------------------------------")
print(df.index.year)
print("\n" + "------------------------------------------------------------------")
print("---------------------------------MONTHS----------------------------------")
print(df.index.month)
print("\n" + "------------------------------------------------------------------")
print("---------------------------------MONTH NAME -----------------------------")
print(df.index.month_name())
print("\n" + "------------------------------------------------------------------")
print("---------------------------------PRICES---------------------------------")
print(df.values[:,0])
print("-------------------------------------------------------------------------")

#### ANNUAL AVERAGE

In [ ]:
# Average gold price per year trend since 1950
Yearly_Sum_df = df.resample('A').mean()
Yearly_Sum_df.plot();
plt.title("Average Gold price (Yearly) since 1950")
plt.xlabel("Year")
plt.ylabel("Price")
plt.grid()
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

#### PER DECADE AVERAGE

In [ ]:
# Average gold price per decade trend since 1950
Decade_Sum_df = df.resample('10Y').mean()
Decade_Sum_df.plot();
plt.title("Average Gold price (Decade) since 1950")
plt.xlabel("Decade")
plt.ylabel("Price")
plt.grid()
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
df

#### TIME SERIES ANALYSIS

#### CHECK WHETHER THE DATA SET OR TIME SERIES IS RANDOM OR NOT

In [ ]:
#CHECK WHETHER THE DATA SET OR TIME SERIES IS RANDOM OR NOT
lag_plot(df)
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=15, rotation=0, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(10,7))
plt.plot(df["price"])
plt.title('Randomness')
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

#### IDENTIFY THE COMPONENTS IN THE GIVEN DATASET

In [ ]:
result = seasonal_decompose(df["price"], model='multiplicative', period=12)
trend = result.trend.dropna()
seasonal = result.seasonal.dropna()
residual = result.resid.dropna()
 
# Plot the decomposed components
# Create figure
fig, (ax1,ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, figsize = (6,7), tight_layout = True)

# Add subplots
ax1.plot(df["price"])
ax2.plot(trend)
ax3.plot(seasonal)
ax4.plot(residual)

# Add Labels for subplots
def get_axis_limits(ax, scale=.82):
    return ax.get_xlim()[1]*scale, ax.get_ylim()[1]*scale

ax1.annotate('Original Series', xy=get_axis_limits(ax1))
ax2.annotate('Trend', xy=get_axis_limits(ax2))
ax3.annotate('Seasonal', xy=get_axis_limits(ax3))
ax4.annotate('Residuals', xy=get_axis_limits(ax4))

# Show texts
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=15, rotation=5, ha='center', va='center', transform=plt.gca().transAxes)
ax1.text(0.5, 0.5, copyright, alpha=0.3, fontsize=15, rotation=5, ha='center', va='center', transform=ax1.transAxes)
ax2.text(0.5, 0.5, copyright, alpha=0.3, fontsize=15, rotation=5, ha='center', va='center', transform=ax2.transAxes)
ax3.text(0.5, 0.5, copyright, alpha=0.3, fontsize=15, rotation=0, ha='center', va='center', transform=ax3.transAxes)
ax4.text(0.5, 0.5, copyright, alpha=0.3, fontsize=15, rotation=0, ha='center', va='center', transform=ax4.transAxes)
plt.show()

#### TEST FOR STATIONARITY

In [ ]:
df["price"]

#### AUGMENTED DICKEY FULLER - ADF Test FOR STATIONARITY

In [ ]:
adfuller_result = adfuller(df["price"], autolag='AIC')
print("----------------------------------ADAFULLER RESULTS----------------------------------")
print(f'ADF Statistic: {adfuller_result[0]}')
print(f'n_lags: {adfuller_result[2]}')
print(f'p-value: {adfuller_result[1]}')
for key, value in adfuller_result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')
print(f'Result: The series is {"not " if adfuller_result[1] < 0.05 else ""}stationary')
print("-------------------------------------------------------------------------------------")

#### KWIATOWSKI-PHILLIPS-SCHMIDT-SHIN - KPSS test around a deterministic trend

In [ ]:
def kpss_test(series, **kw):    
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print("----------------------------------KPSS RESULTS----------------------------------")
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')
    print("--------------------------------------------------------------------------------")
    
kpss_test(df["price"].dropna(), regression='ct')

#### AUTO CORRELATION

In [ ]:
#AUTO CORRELATION
plot_acf(df["price"].dropna())
plt.show()

#### PARTIAL AUTO CORRELATYION

In [ ]:
#PARTIAL AUTO CORRELATYION
plot_acf(df["price"].dropna(), lags=30)
plt.show()

#### TIME SERIES FORECASTING - MODELS

#### CREATE TRAIN TEST SPLIT

In [ ]:
train = df[df.index.year <= 2015] 
test = df[df.index.year > 2015]

print(train.shape)
print(test.shape)

In [ ]:
train['price'].plot(figsize=(13,5), fontsize=14)
test['price'].plot(figsize=(13,5), fontsize=14)
plt.legend(['Training Data','Test Data'])
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

#### LINEAR REGRESSION

In [ ]:
train_time = [i+1 for i in range(len(train))]
test_time = [i+len(train)+1 for i in range(len(test))]
len(train_time), len(test_time)

In [ ]:
Linear_Regression_train = train.copy()
Linear_Regression_test = test.copy()

Linear_Regression_train['time'] = train_time
Linear_Regression_test['time'] = test_time

Linear_Regression_Model = LinearRegression()
Linear_Regression_Model.fit(Linear_Regression_train[['time']],Linear_Regression_train['price'].values)

Linear_Regression_test_predictions  = Linear_Regression_Model.predict(Linear_Regression_test[['time']])
Linear_Regression_test['forecast'] = Linear_Regression_test_predictions

plt.figure(figsize=(13,6))
plt.plot( train['price'], label='Train')
plt.plot(test['price'], label='Test')
plt.plot(Linear_Regression_test['forecast'], label='Regression On Time_Test Data')
plt.legend(loc='best')
plt.grid()
plt.title("Linear Rgeression Model Forecast")
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

#### Mean absolute percentage error - MAPE
#### MAPE is a metric that defines the accuracy of a forecasting method. 
#### It represents the average of the absolute percentage errors of each entry in a dataset.
#### A MAPE less than 5% is considered as an indication that the forecast is acceptably accurate. 
#### A MAPE greater than 10% but less than 25% indicates low, but acceptable accuracy and 
#### MAPE greater than 25% very low accuracy.

In [ ]:
def mape(actual,pred):
    return round((np.mean(abs(actual-pred)/actual))*100,2)

In [ ]:
# Get MAPE of the model

mape_Linear_Regression_test = mape(test['price'].values,Linear_Regression_test_predictions)
print("For the Linear Regression On Time forecast on the Test Data,  MAPE is %3.3f" %(mape_Linear_Regression_test),"%")

In [ ]:
results = pd.DataFrame({'Test MAPE (%)': [mape_Linear_Regression_test]},index=['Linear Regression On Time'])
results

#### NAIVE 

In [ ]:
Naive_train = train.copy()
Naive_test = test.copy()

Naive_test['naive'] = np.asarray(train['price'])[len(np.asarray(train['price']))-1]
Naive_test['naive'].head()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(Naive_train['price'], label='Train')
plt.plot(test['price'], label='Test')
plt.plot(Naive_test['naive'], label='Naive Forecast on Test Data')
plt.legend(loc='best')
plt.title("Naive Forecast")
plt.grid()
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
# Get MAPE of the model

mape_naive_test = mape(test['price'].values,Naive_test['naive'].values)
print("For the Naive forecast on the Test Data,  MAPE is %3.3f" %(mape_naive_test),"%")

In [ ]:
resultsDf_2 = pd.DataFrame({'Test MAPE (%)': [mape_naive_test]},index=['Naive Model'])

results = pd.concat([results, resultsDf_2])
results

#### SIMPLE MOVING AVERAGE

In [ ]:
SimpleAvg_train = train.copy()
SimpleAvg_test = test.copy()
SimpleAvg_test['mean_forecast'] = train['price'].mean()
SimpleAvg_test.head()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(SimpleAvg_train['price'], label='Train')
plt.plot(SimpleAvg_test['price'], label='Test')
plt.plot(SimpleAvg_test['mean_forecast'], label='Simple Average on Test Data')
plt.legend(loc='best')
plt.title("Simple Moving Average Forecast")
plt.grid()
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
## Test Data - MAPE

mape_simple_average_test = mape(test['price'].values,SimpleAvg_test['mean_forecast'].values)
print("For the Simple Average forecast on the Test Data,  MAPE is %3.3f" %(mape_simple_average_test),"%")

In [ ]:
resultsDf_3 = pd.DataFrame({'Test MAPE (%)': [mape_simple_average_test]},index=['Simple Average Model'])

results = pd.concat([results, resultsDf_3])
results

#### MOVING AVERAGE

In [ ]:
Mvg_Avg = df.copy()
Mvg_Avg['Trailing_2'] = Mvg_Avg['price'].rolling(2).mean()
Mvg_Avg['Trailing_3'] = Mvg_Avg['price'].rolling(3).mean()
Mvg_Avg['Trailing_5'] = Mvg_Avg['price'].rolling(5).mean()
Mvg_Avg['Trailing_7'] = Mvg_Avg['price'].rolling(7).mean()
Mvg_Avg.head()

In [ ]:
## Plotting on the whole data

plt.figure(figsize=(16,8))
plt.plot(Mvg_Avg['price'], label='Train')
plt.plot(Mvg_Avg['Trailing_2'],label='2 Point Moving Average')
plt.plot(Mvg_Avg['Trailing_3'],label='3 Point Moving Average')
plt.plot(Mvg_Avg['Trailing_5'],label = '5 Point Moving Average')
plt.plot(Mvg_Avg['Trailing_7'],label = '7 Point Moving Average')
plt.title("Moving Average Forecast")
plt.legend(loc = 'best')
plt.grid()
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

#### TRAILING MOVING AVERAGE

In [ ]:
#Creating train and test set 
trailing_Mvg_Avg_train=Mvg_Avg[Mvg_Avg.index.year <= 2015] 
trailing_Mvg_Avg_test=Mvg_Avg[Mvg_Avg.index.year > 2015]

In [ ]:
## Plotting on both the Training and Test data

plt.figure(figsize=(16,8))
plt.plot(trailing_Mvg_Avg_train['price'], label='Train')
plt.plot(trailing_Mvg_Avg_test['price'], label='Test')

plt.plot(trailing_Mvg_Avg_train['Trailing_2'],label='2 Point Trailing Moving Average on Training Set')
plt.plot(trailing_Mvg_Avg_train['Trailing_3'],label='3 Point Trailing Moving Average on Training Set')
plt.plot(trailing_Mvg_Avg_train['Trailing_5'],label = '5 Point Trailing Moving Average on Training Set')
plt.plot(trailing_Mvg_Avg_train['Trailing_7'],label = '7 Point Trailing Moving Average on Training Set')

plt.plot(trailing_Mvg_Avg_test['Trailing_2'], label='2 Point Trailing Moving Average on Test Set')
plt.plot(trailing_Mvg_Avg_test['Trailing_3'], label='3 Point Trailing Moving Average on Test Set')
plt.plot(trailing_Mvg_Avg_test['Trailing_5'],label = '5 Point Trailing Moving Average on Test Set')
plt.plot(trailing_Mvg_Avg_test['Trailing_7'],label = '7 Point Trailing Moving Average on Test Set')
plt.legend(loc = 'best')
plt.grid()
plt.title("Moving Forecast")
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
## Test Data - MAPE --> 2 point Trailing MA

mape_moving_average_test_2 = mape(test['price'].values,trailing_Mvg_Avg_test['Trailing_2'].values)
print("For 2 point Moving Average Model forecast on the Training Data,  MAPE is %3.3f" %(mape_moving_average_test_2),"%")

## Test Data - MAPE  --> 3 point Trailing MA

mape_moving_average_test_3 = mape(test['price'].values,trailing_Mvg_Avg_test['Trailing_3'].values)
print("For 3 point Moving Average Model forecast on the Training Data,  MAPE is %3.3f" %(mape_moving_average_test_3),"%")

## Test Data - MAPE --> 5 point Trailing MA

mape_moving_average_test_5 = mape(test['price'].values,trailing_Mvg_Avg_test['Trailing_5'].values)
print("For 5 point Moving Average Model forecast on the Training Data,  MAPE is %3.3f" %(mape_moving_average_test_5),"%")

## Test Data - MAPE  --> 7 point Trailing MA

mape_moving_average_test_7 = mape(test['price'].values,trailing_Mvg_Avg_test['Trailing_7'].values)
print("For 7 point Moving Average Model forecast on the Training Data,  MAPE is %3.3f " %(mape_moving_average_test_7),"%")

In [ ]:
resultsDf_4 = pd.DataFrame({'Test MAPE (%)': [mape_moving_average_test_2,mape_moving_average_test_3
                                          ,mape_moving_average_test_5,mape_moving_average_test_7]}
                           ,index=['2 point Trailing Moving Average','3 point Trailing Moving Average'
                                   ,'5 point Trailing Moving Average','7 point Trailing Moving Average'])

results = pd.concat([results, resultsDf_4])
results

#### SIMPLE EXPONENTIAL SMOOTHING

In [ ]:
Simple_Exponential_Smoothing_train = train.copy()
Simple_Exponential_Smoothing_test = test.copy()

model_Simple_Exponential_Smoothing = SimpleExpSmoothing(Simple_Exponential_Smoothing_train['price'])
model_Simple_Exponential_Smoothing_autofit = model_Simple_Exponential_Smoothing.fit(optimized=True)

model_Simple_Exponential_Smoothing_autofit.params

In [ ]:
Simple_Exponential_Smoothing_test['predict'] = model_Simple_Exponential_Smoothing_autofit.forecast(steps=len(test))
Simple_Exponential_Smoothing_test.head()

In [ ]:
## Plotting on both the Training and Test data

plt.figure(figsize=(16,8))
plt.plot(Simple_Exponential_Smoothing_train['price'], label='Train')
plt.plot(Simple_Exponential_Smoothing_test['price'], label='Test')

plt.plot(Simple_Exponential_Smoothing_test['predict'], label='Alpha = 0.995 Simple Exponential Smoothing predictions on Test Set')

plt.legend(loc='best')
plt.grid()
plt.title('Alpha = 0.995 Simple Exponential Smoothing Forecast')
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
## Test Data

mape_simple_moving_Exponential_Smoothing_test_1 = mape(Simple_Exponential_Smoothing_test['price'].values,Simple_Exponential_Smoothing_test['predict'].values)
print("For Alpha = 0.995 Simple Exponential Smoothing Model forecast on the Test Data, MAPE is %3.3f" %(mape_simple_moving_Exponential_Smoothing_test_1),"%")

In [ ]:
resultsDf_5 = pd.DataFrame({'Test MAPE (%)': [mape_simple_moving_Exponential_Smoothing_test_1]},index=['Alpha=0.995, Simple Exponential Smoothing'])

results = pd.concat([results, resultsDf_5])
results

In [ ]:
resultsDf_6 = pd.DataFrame({'Alpha Values':[],'Train MAPE':[],'Test MAPE': []})

for i in np.arange(0.3,1,0.1):
    model_Simple_Exponential_Smoothing_alpha_i = model_Simple_Exponential_Smoothing.fit(smoothing_level=i,optimized=False,use_brute=True)
    Simple_Exponential_Smoothing_train['predict',i] = model_Simple_Exponential_Smoothing_alpha_i.fittedvalues
    Simple_Exponential_Smoothing_test['predict',i] = model_Simple_Exponential_Smoothing_alpha_i.forecast(steps=55)
    
    mape_model_Simple_Exponential_Smoothing_train_i = mape(Simple_Exponential_Smoothing_train['price'].values,Simple_Exponential_Smoothing_train['predict',i].values)
    
    mape_model_Simple_Exponential_Smoothing_test_i = mape(Simple_Exponential_Smoothing_test['price'].values,Simple_Exponential_Smoothing_test['predict',i].values)

    resultsDf_6 = pd.DataFrame({'Alpha Values':[i],'Train MAPE':[mape_model_Simple_Exponential_Smoothing_train_i] 
                                      ,'Test MAPE':[mape_model_Simple_Exponential_Smoothing_test_i]})
    #resultsDf_6 = resultsDf_6.append({'Alpha Values':i,'Train MAPE':mape_model_Simple_Exponential_Smoothing_train_i 
     #                                 ,'Test MAPE':mape_model_Simple_Exponential_Smoothing_test_i}, ignore_index=True)

In [ ]:
resultsDf_6.sort_values(by=['Test MAPE'],ascending=True)

In [ ]:
## Plotting on both the Training and Test data

plt.figure(figsize=(18,9))
plt.plot(Simple_Exponential_Smoothing_train['price'], label='Train')
plt.plot(Simple_Exponential_Smoothing_test['price'], label='Test')
plt.plot(Simple_Exponential_Smoothing_test['predict'], label='Alpha = 1 Simple Exponential Smoothing predictions on Test Set')
plt.plot(Simple_Exponential_Smoothing_test['predict', 0.3], label='Alpha = 0.3 Simple Exponential Smoothing predictions on Test Set')

plt.legend(loc='best')
plt.grid()
plt.title('Alpha = 0.3 Simple Exponential Smoothing Forecast')
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
resultsDf_6_1 = pd.DataFrame({'Test MAPE (%)': [resultsDf_6.sort_values(by=['Test MAPE'],ascending=True).values[0][2]]}
                           ,index=['Alpha=0.3, Simple Exponential Smoothing'])

results = pd.concat([results, resultsDf_6_1])
results

#### DOUBLE EXPONENTIAL SMOOTHING

In [ ]:
Double_Exponential_Smoothing_train = train.copy()
Double_Exponential_Smoothing_test = test.copy()

model_Double_Exponential_Smoothing = Holt(Double_Exponential_Smoothing_train['price'])

resultsDf_7 = pd.DataFrame({'Alpha Values':[],'Beta Values':[],'Train MAPE':[],'Test MAPE': []})

for i in np.arange(0.3,1.1,0.1):
    for j in np.arange(0.3,1.1,0.1):
        model_Double_Exponential_Smoothing_alpha_i_j = model_Double_Exponential_Smoothing.fit(smoothing_level=i,smoothing_trend=j,optimized=False,use_brute=True)
        Double_Exponential_Smoothing_train['predict',i,j] = model_Double_Exponential_Smoothing_alpha_i_j.fittedvalues
        Double_Exponential_Smoothing_test['predict',i,j] = model_Double_Exponential_Smoothing_alpha_i_j.forecast(steps=55)
        
        mape_Double_Exponential_Smoothing_train = mape(Double_Exponential_Smoothing_train['price'].values,Double_Exponential_Smoothing_train['predict',i,j].values)
        
        mape_Double_Exponential_Smoothing_test = mape(Double_Exponential_Smoothing_test['price'].values,Double_Exponential_Smoothing_test['predict',i,j].values)

        resultsDf_7 = pd.DataFrame({'Alpha Values':[i],'Beta Values':[j],'Train MAPE':[mape_Double_Exponential_Smoothing_train]
                                          ,'Test MAPE':[mape_Double_Exponential_Smoothing_test]}) 
        #resultsDf_7 = resultsDf_7.append({'Alpha Values':i,'Beta Values':j,'Train MAPE':mape_Double_Exponential_Smoothing_train
         #                                 ,'Test MAPE':mape_Double_Exponential_Smoothing_test}, ignore_index=True)

resultsDf_7.sort_values(by=['Test MAPE']).head()

In [ ]:
## Plotting on both the Training and Test data

plt.figure(figsize=(18,9))
plt.plot(Double_Exponential_Smoothing_train['price'], label='Train')
plt.plot(Double_Exponential_Smoothing_test['price'], label='Test')

plt.plot(Double_Exponential_Smoothing_test['predict', 0.3, 0.3], label='Alpha=0.3, Beta=0.3, Double Exponential Smoothing predictions on Test Set')


plt.legend(loc='best')
plt.grid()
plt.title('Alpha = 0.3, Beta=0.3, Double Exponential Smoothing Forecast')
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
resultsDf_7_1 = pd.DataFrame({'Test MAPE (%)': [resultsDf_7.sort_values(by=['Test MAPE']).values[0][3]]}
                           ,index=['Alpha=0.3, Beta=0.3, Double Exponential Smoothing'])

results = pd.concat([results, resultsDf_7_1])
results

#### TRIPLE SMOOTHING

In [ ]:
Triple_Exponential_Smoothing_train = train.copy()
Triple_Exponential_Smoothing_test = test.copy()

model_Triple_Exponential_Smoothing = ExponentialSmoothing(Triple_Exponential_Smoothing_train['price'],trend='additive',freq='Y')
model_Triple_Exponential_Smoothing_autofit = model_Triple_Exponential_Smoothing.fit()

model_Triple_Exponential_Smoothing_autofit.params

In [ ]:
## Prediction on the test data

Triple_Exponential_Smoothing_test['auto_predict'] = model_Triple_Exponential_Smoothing_autofit.forecast(steps=len(test))
Triple_Exponential_Smoothing_test.head()

In [ ]:
## Plotting on both the Training and Test using autofit

plt.figure(figsize=(18,9))
plt.plot(Triple_Exponential_Smoothing_train['price'], label='Train')
plt.plot(Triple_Exponential_Smoothing_test['price'], label='Test')

plt.plot(Triple_Exponential_Smoothing_test['auto_predict'], label='Alpha=0.99, Beta=0.05, Gamma=0.001, Triple Exponential Smoothing predictions on Test Set')

plt.legend(loc='best')
plt.grid()
plt.title('Alpha = 0.99 Beta=0.05, Gamma=0.001 Triple Exponential Smoothing Forecast')
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
## Test Data

mape_Triple_Exponential_Smoothing_test_1 = mape(Triple_Exponential_Smoothing_test['price'].values, Triple_Exponential_Smoothing_test['auto_predict'].values)
print("For A=0.99,B=0.05,G=0.001, Triple Exponential Smoothing Model forecast on the Test Data,  MAPE is %3.3f" %(mape_Triple_Exponential_Smoothing_test_1),"%")

In [ ]:
resultsDf_8_1 = pd.DataFrame({'Test MAPE (%)': [mape_Triple_Exponential_Smoothing_test_1]}
                           ,index=['Alpha=0.99, Beta=0.05, Gamma=0.001, Triple Exponential Smoothing'])

results = pd.concat([results, resultsDf_8_1])
results

In [ ]:
## First we will define an empty dataframe to store our values from the loop

resultsDf_8_2 = pd.DataFrame({'Alpha Values':[],'Beta Values':[],'Gamma Values':[],'Train MAPE':[],'Test MAPE': []})
resultsDf_8_2

In [ ]:
for i in np.arange(0.3,1.1,0.1):
    for j in np.arange(0.3,1.1,0.1):
        for k in np.arange(0.3,1.1,0.1):
            model_Triple_Exponential_Smoothing_alpha_i_j_k = model_Triple_Exponential_Smoothing.fit(smoothing_level=i,smoothing_trend=j,smoothing_seasonal=k,optimized=False,use_brute=True)
            Triple_Exponential_Smoothing_train['predict',i,j,k] = model_Triple_Exponential_Smoothing_alpha_i_j_k.fittedvalues
            Triple_Exponential_Smoothing_test['predict',i,j,k] = model_Triple_Exponential_Smoothing_alpha_i_j_k.forecast(steps=55)
        
            mape_model_Triple_Exponential_Smoothing_train = mape(Triple_Exponential_Smoothing_train['price'].values,Triple_Exponential_Smoothing_train['predict',i,j,k].values)
            
            mape_model_Triple_Exponential_Smoothing_test = mape(Triple_Exponential_Smoothing_test['price'].values,Triple_Exponential_Smoothing_test['predict',i,j,k].values)

            resultsDf_8_2 = pd.DataFrame({'Alpha Values':[i],'Beta Values':[j],'Gamma Values':[k],
                                                  'Train MAPE':[mape_model_Triple_Exponential_Smoothing_train],'Test MAPE':[mape_model_Triple_Exponential_Smoothing_test]})
            
            #resultsDf_8_2 = resultsDf_8_2.append({'Alpha Values':i,'Beta Values':j,'Gamma Values':k,
             #                                     'Train MAPE':mape_model_Triple_Exponential_Smoothing_train,'Test MAPE':mape_model_Triple_Exponential_Smoothing_test}

In [ ]:
resultsDf_8_2.sort_values(by=['Test MAPE']).head()

In [ ]:
model_Triple_Exponential_Smoothing_alpha_best = model_Triple_Exponential_Smoothing.fit(smoothing_level=0.4,
                                      smoothing_trend=0.3,
                                      smoothing_seasonal=0.6,
                                      optimized=False,
                                      use_brute=True)
Triple_Exponential_Smoothing_train['predict',0.4,0.3,0.6] = model_Triple_Exponential_Smoothing_alpha_best.fittedvalues
Triple_Exponential_Smoothing_test['predict',0.4,0.3,0.6] = model_Triple_Exponential_Smoothing_alpha_best.forecast(steps=55)

In [ ]:
## Plotting on both the Training and Test data using brute force alpha, beta and gamma determination

plt.figure(figsize=(18,9))
plt.plot(Triple_Exponential_Smoothing_train['price'], label='Train')
plt.plot(Triple_Exponential_Smoothing_test['price'], label='Test')

#The value of alpha and beta is taken like that by python
plt.plot(Triple_Exponential_Smoothing_test['predict', 0.4, 0.3, 0.6], label='Alpha=0.4, Beta=0.3, Gamma=0.6, Triple Exponential Smoothing predictions on Test Set')


plt.legend(loc='best')
plt.grid()
plt.title('Alpha = 0.4 Beta=0.3, Gamma=0.6 Triple Exponential Smoothing Forecast')
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
## Test Data

mape_Triple_Exponential_Smoothing_test = mape(Triple_Exponential_Smoothing_test['price'].values,Triple_Exponential_Smoothing_test['predict',0.4,0.3,0.6].values)
print("For A=0.4,B=0.3,G=0.6, Triple Exponential Smoothing Model forecast on the Test Data,  MAPE is %3.3f" %(mape_Triple_Exponential_Smoothing_test),"%")

In [ ]:
resultsDf_9_1 = pd.DataFrame({'Test MAPE (%)': [mape_Triple_Exponential_Smoothing_test]}
                           ,index=['Alpha=0.4, Beta=0.3, Gamma=0.6, Triple Exponential Smoothing'])

results = pd.concat([results, resultsDf_9_1])
results

#### EXPONENTIAL SMOOTHING MODEL @ Alpha=0.4, Beta=03 and Gamma=0.6

In [ ]:
Triple_Exponential_Smoothing_model =  ExponentialSmoothing(df, trend='additive')

Triple_Exponential_Smoothing_autofit = Triple_Exponential_Smoothing_model.fit(smoothing_level=0.4, smoothing_trend=0.3, smoothing_seasonal=0.6)


Triple_Exponential_Smoothing_autofit.params

In [ ]:
MAPE_Triple_Exponential_Smoothing_model = mape(df['price'].values,Triple_Exponential_Smoothing_autofit.fittedvalues)

print('MAPE:',MAPE_Triple_Exponential_Smoothing_model)

In [ ]:
# Getting the predictions for the same number of times stamps that are present in the test data
prediction = Triple_Exponential_Smoothing_autofit.forecast(steps=len(test))

In [ ]:
# Compute 95% confidence interval for predicted values
pred_df = pd.DataFrame({'lower_CI':prediction - 1.96*np.std(Triple_Exponential_Smoothing_autofit.resid,ddof=1),
                        'prediction':prediction,
                        'upper_CI': prediction + 1.96*np.std(Triple_Exponential_Smoothing_autofit.resid,ddof=1)})
pred_df.head()

In [ ]:
# plot the forecast along with the confidence band

axis = df.plot(label='Actual', figsize=(15,8))
pred_df['prediction'].plot(ax=axis, label='Forecast', alpha=0.5)
axis.fill_between(pred_df.index, pred_df['lower_CI'], pred_df['upper_CI'], color='k', alpha=.15)
axis.set_xlabel('Year-Months')
axis.set_ylabel('price')
plt.legend(loc='best')
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

#### ARIMA MODEL

In [ ]:
#ARIMA MODEL
model = ARIMA(df["price"], order=(0, 1, 1)) 
results_ARIMA = model.fit()

summary = results_ARIMA.summary()
print(summary)

fig, ax = plt.subplots()
ax = df['price'].plot(ax=ax)
plot_predict(results_ARIMA, ax=ax)
plt.title("ARIIMA Model Actual VS Forecast")
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=15, rotation=0, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

#### MODEL DIAGNOSTICS

In [ ]:
# plot residual errors
residuals = results_ARIMA.resid
residuals.plot()
residuals.plot(kind='kde')
plt.title("ARIMA Residual errors")
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=15, rotation=0, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
# EVAULATE THE MODEL
print("\n" + "---------------------------AIC and BIC---------------------------")
print(f"AIC: {results_ARIMA.aic}")
print(f"BIC: {results_ARIMA.bic}")
print("\n" + "-----------------------------------------------------------------")

In [ ]:
#FORECAST
arima_train_data = train.copy()
arima_test_data = test.copy()

# Fit the model to training data. Replace p, d, q with our ARIMA parameters
model = ARIMA(arima_train_data, order=(0, 1, 1))  
result = model.fit()

## Forecast
forecast = result.forecast(steps=len(arima_test_data))


#forecast = results_ARIMA.forecast(3)
print("\n" + "---------------------------FIITED ARIMA MODEL NEXT FORECAST RESULT---------------------------")
print(forecast)
print("----------------------------------------------------------------------------------------------------")

In [ ]:
##Evaluate model statistics
## Evaluate model performance on the test set
rmse = mean_squared_error(arima_test_data, forecast, squared=False)
print(f"ARIMA ROOT MEAN SQAURED ERROR: {rmse}")

In [ ]:
# Visualize our time series
plt.figure(figsize=(18,9))
plt.plot(arima_train_data, label='Train', color='blue')
plt.plot(arima_test_data, label='Test', color='green')
plt.plot(forecast, label='Forecast', color='red')

plt.legend(loc='best')
plt.grid()
plt.title('ARIMA Forecast vs Actual')
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()


# plot predictions and actual values 
#arima_train_data.plot(legend = True) 
#arima_test_data.plot(legend = True) 
#forecast.plot(legend = True) 

#### USE AUTO ARIMA

In [ ]:
# Fit auto_arima function to AirPassengers dataset 
stepwise_fit = auto_arima(df["price"], start_p = 1, start_q = 1, 
                          max_p = 3, max_q = 3, m = 12, 
                          start_P = 0, seasonal = True, 
                          d = None, D = 1, trace = True, 
                          error_action ='ignore',   # we don't want to know if an order does not work 
                          suppress_warnings = True,  # we don't want convergence warnings 
                          stepwise = True)           # set to stepwise 
  
# To print the summary 
stepwise_fit.summary()

# Fit ARIMA Model to dataset
# Split data into train / test sets 
auto_arima_train_data = train.copy()
auto_arima_test_data = test.copy()


# Fit a SARIMAX(0, 1, 1)x(2, 1, 1, 12) on the training set 
model = SARIMAX(auto_arima_train_data['price'], order = (0, 1, 1), seasonal_order =(2, 1, 1, 12)) 

result = model.fit() 
auto_arima_result = result.summary() 
print(auto_arima_result)

In [ ]:
#Predictions of ARIMA Model against the test set
start = len(auto_arima_train_data) 
end = len(auto_arima_train_data) + len(auto_arima_test_data) - 1

# Predictions against the test set 
predictions = result.predict(start, end, typ = 'levels').rename("Predictions") 

# plot predictions and actual values 
predictions.plot(legend = True) 
df['price'].plot(legend = True) 
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()

In [ ]:
#Evaluate the model using MSE and RMSE

# Calculate root mean squared error 
Auto_Arima_RMSE = mean_squared_error(auto_arima_test_data, predictions, squared=False) 
print(f"AUTO ARIMA ROOT MEAN SQAURED ERROR: {Auto_Arima_RMSE}")

# Calculate mean squared error 
Auto_ARIMA_mse = mean_squared_error(auto_arima_test_data, predictions) 
print(f"AUTO ARIMA MEAN SQAURED ERROR: {Auto_ARIMA_mse}")

In [ ]:
#Forecast using ARIMA Model
# Train the model on the full dataset 
model = model = SARIMAX(df['price'], 
                        order = (0, 1, 1), 
                        seasonal_order =(2, 1, 1, 12)) 
result = model.fit() 

# Forecast for the next 3 years 
forecast = result.predict(start = len(df), 
                        end = (len(df)-1) + 3 * 12, 
                        typ = 'levels').rename('Forecast') 

# Plot the forecast values 
df['price'].plot(figsize = (12, 5), legend = True) 
forecast.plot(legend = True)
copyright = "\u00A9" + " " + str(datetime.today().year) + " " + "JM Consulting"
plt.text(0.5, 0.5, copyright, alpha=0.3, fontsize=25, rotation=25, ha='center', va='center', transform=plt.gca().transAxes)
plt.show()